In [2]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 1.1 MB/s 


In [3]:
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten, TimeDistributed,Dropout,GRU,Bidirectional,RepeatVector,LSTM, Conv1D, GlobalMaxPool1D, GlobalAveragePooling1D, AveragePooling1D, MaxPooling1D, SpatialDropout1D
from keras.losses import sparse_categorical_crossentropy
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
import pickle
import tensorflow as tf
import os
import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,  Attention, Input, Normalization
from tensorflow.keras.layers import concatenate, average, multiply, Reshape, dot
from tensorflow.keras.layers import Multiply, Add, Subtract, Dot, Average, Maximum, Concatenate
from numpy import dot
from numpy.linalg import norm
from numpy import sqrt

from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
from scipy.sparse import csr_matrix
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

In [ ]:
from pathlib import Path
path = Path(__file__).parent.parent

In [4]:
food = pd.read_excel('/content/drive/MyDrive/Food Recommend Project/data/Food_description.xlsx')
food.sample(3)

,FoodID,Tên món ăn,Vùng miền,Thành phần,"Thể loại (nướng, nước, khô, ...)",mô tả ngắn gọn ( <100 từ)
15,F16,Bún mắm,Miền Nam,"Mắm cá linh, mắm cá sặc, tôm, mực, thịt lợn, x...",Nước,Bún mắm là một trong số các món ăn đặc sản của...
12,F13,Bánh canh chả cá,Miền Trung,"sợi bánh canh bột lọc, chả cá chiên, chả cá dầ...",Nước,Bánh canh chả cá chính là món ăn độc nhất của ...
20,F21,Cơm gà xối mỡ,Miền Nam,"Thịt gà, trứng gà, cơm, mỡ hành, dưa leo và mộ...",Khô,Cơm gà xối mỡ là món quen thuộc với người dân ...


In [5]:
train_rating = pd.read_csv("/content/drive/MyDrive/Food Recommend Project/data/training_data_new.csv")
train_rating.head()

,UserID,FoodId,rating
0,U112,F24,4.0
1,U130,F26,4.0
2,U120,F5,2.0
3,U170,F12,3.0
4,U92,F30,4.0


In [6]:
df_food = train_rating.pivot(
    index = 'FoodId',
    columns = 'UserID',
    values = 'rating'
).fillna(0)

df_food

UserID,U1,U10,U100,U101,U102,U103,U104,U105,U106,U107,U108,U109,U11,U110,U111,U112,U113,U114,U115,U116,U117,U118,U119,U12,U120,U121,U122,U123,U124,U125,U126,U127,U128,U129,U13,U130,U131,U132,U133,U134,...,U63,U64,U65,U66,U67,U68,U69,U7,U70,U71,U72,U73,U74,U75,U76,U77,U78,U79,U8,U80,U81,U82,U83,U84,U85,U86,U87,U88,U89,U9,U90,U91,U92,U93,U94,U95,U96,U97,U98,U99
FoodId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
F1,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,5.0,4.0,0.0,0.0,4.0,2.0,5.0,5.0,0.0,3.0,3.0,3.0,0.0,5.0,0.0,...,0.0,3.0,4.0,5.0,3.0,2.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,2.0,3.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,5.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
F10,0.0,0.0,5.0,0.0,2.0,4.0,5.0,0.0,4.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0,5.0,0.0,4.0,5.0,0.0,5.0,4.0,0.0,4.0,4.0,2.0,2.0,4.0,0.0,5.0,3.0,4.0,0.0,5.0,0.0,...,5.0,3.0,5.0,4.0,5.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,2.0,3.0,4.0,4.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
F11,4.0,4.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,4.0,4.0,4.0,5.0,2.0,5.0,4.0,0.0,5.0,3.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,3.0,0.0,4.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,2.0,0.0,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,2.0,0.0,5.0,3.0,3.0,4.0,0.0,0.0,0.0,5.0,0.0,5.0,3.0,5.0,4.0,0.0,4.0,4.0
F12,0.0,3.0,3.0,0.0,0.0,0.0,4.0,3.0,3.0,0.0,5.0,4.0,3.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,3.0,3.0,5.0,0.0,5.0,4.0,0.0,2.0,5.0,2.0,5.0,0.0,3.0,0.0,3.0,1.0,3.0,5.0,4.0,...,0.0,0.0,3.0,5.0,5.0,3.0,0.0,5.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,1.0,3.0,5.0,0.0,0.0,4.0,0.0,5.0,0.0,3.0,0.0,4.0,5.0,0.0,4.0,4.0,3.0,4.0,3.0,3.0,0.0,5.0,3.0,1.0
F13,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,5.0,5.0,0.0,5.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,4.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
F14,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
F15,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,5.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,0.0,0.0,0.0,5.0,0.0,4.0,4.0,4.0,0.0,0.0,4.0,5.0
F16,0.0,4.0,0.0,2.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,4.0,4.0,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,3.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,4.0,0.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,3.0,0.0,3.0,3.0,1.0,0.0,0.0,3.0,5.0
F17,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0


In [7]:
cleaned_user = pd.read_csv("/content/drive/MyDrive/Food Recommend Project/data/cleaned_user.csv")
test_user = pd.read_csv("/content/drive/MyDrive/Food Recommend Project/data/test_user.csv")
cleaned_user = pd.concat([cleaned_user, test_user])
cleaned_user.tail()

,UserID,Age,Weight,Height,like_all,like_mon_nuoc,like_mon_kho,like_mon_nuong,like_mon_cay,is_north,is_south,is_middle,gender_others,gender_male,gender_female
0,U236,22.0,3.0,3.0,0,1,1,1,0,1,0,0,0,1,0
1,U237,23.0,3.0,3.0,0,1,0,1,0,0,0,1,0,1,0
2,U238,21.0,4.0,4.0,1,0,0,0,0,0,1,0,0,1,0
3,U239,21.0,4.0,3.0,0,1,1,1,0,0,1,0,0,1,0
4,U240,21.0,5.0,5.0,0,1,0,0,0,1,0,0,0,1,0


In [8]:
food_description = pd.read_csv("/content/drive/MyDrive/Food Recommend Project/data/food_description.csv")
food_description.head()

,FoodID,processed_description
0,F1,gạo nếp đậu xanh vừng đậu phộng xôi gạo nếp hấ...
1,F2,mìnước lèo thịt tôm cá trứng đậu phộng bánh ...
2,F3,bánh phở lèo thịt bò trứng chần hành lá tương ...
3,F4,gạo thịt sườn heo mỡ hành trứng chiên rau chua...
4,F5,miến dong thịt gà nấm rau thơm miến gà vị thịt...


In [9]:
food_list = food_description.FoodID.unique()
food_list

array(['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28', 'F29',
       'F30'], dtype=object)

In [10]:
food_information = pd.read_excel('/content/drive/MyDrive/Food Recommend Project/data/Food_description.xlsx')
food_information.head()

,FoodID,Tên món ăn,Vùng miền,Thành phần,"Thể loại (nướng, nước, khô, ...)",mô tả ngắn gọn ( <100 từ)
0,F1,Xôi,"Miền Bắc, Miền Trung, Miền Nam","Gạo nếp, đậu xanh, Vừng, Đậu phộng",Món khô,Xôi được làm từ gạo nếp hấp chín bằng hơi nước...
1,F2,Mì quảng,Miền Trung,"mì,nước lèo, thịt, tôm, cá, trứng , đậu phộng...",Món nước,"Có nguồn gốc ở Quảng Nam, mì quảng là món mì s..."
2,F3,Phở bò,Miền Bắc,"bánh phở, nước lèo, thịt bò, trứng chần, hành ...",Món nước,"Món ăn nổi tiếng, được nhiều người ưu thích. Ă..."
3,F4,Cơm tấm,Miền Nam,"Gạo tấm, thịt sườn heo, mỡ hành, trứng chiên, ...",Món khô,"Hạt cơm tấm mềm, ăn kèm với thịt sườn nướng rấ..."
4,F5,Miến gà,Miền Nam,"Miến dong, thịt gà, nấm, rau thơm",Món nước,"Miến gà có vị ngọt thanh từ thịt gà, hương thơ..."


In [11]:
food_feature = pd.read_csv("/content/drive/MyDrive/Food Recommend Project/data/food_features.csv")
food_feature.columns = ['FoodID', 'is_north1', 'is_south1', 'is_middle1', 'is_mon_kho1',
       'is_mon_nuoc1', 'is_mon_nuong1']
food_feature.head()

,FoodID,is_north1,is_south1,is_middle1,is_mon_kho1,is_mon_nuoc1,is_mon_nuong1
0,F1,1,1,1,1,0,0
1,F2,0,0,1,0,1,0
2,F3,1,0,0,0,1,0
3,F4,0,1,0,1,0,0
4,F5,0,1,0,0,1,0


In [12]:
vectorizer = pickle.load(open("/content/drive/MyDrive/Food Recommend Project/Linh Models for redcommend/tf_idf.pkl", 'rb'))
tf_idf_matrix = vectorizer.transform(food_description.processed_description).toarray()

In [13]:
def get_food_name(food_id):
    return food_information[food_information.FoodID == food_id]['Tên món ăn'].values[0]

In [14]:
def get_user_context(user_id):
  return cleaned_user[cleaned_user['UserID'] == user_id].iloc[:, 1:]

In [15]:
def food_average_rating(food_id):
    return train_rating[train_rating['FoodId'] == food_id].mean()[0]

In [16]:
def average_rating_recommend(rated_food):
    recommend_dict = {}
    for food_id in food_list:
        if food_id not in rated_food and food_id != 'F22':
            recommend_dict[get_food_name(food_id)] = food_average_rating(food_id)
    recommend_dict['Mì cay'] = 0
    return recommend_dict

# CF

In [17]:
def get_rating(userid, foodid):
    return (train_rating.loc[((train_rating.UserID== userid)&(train_rating.FoodId== foodid)), 'rating'].iloc[0])

In [18]:
def get_foodid(userid): 
    return (train_rating.loc[(train_rating.UserID==userid), 'FoodId'].tolist())

In [19]:
def get_food_title(foodid):
    return (food_information.loc[(food_information.FoodID==foodid), 'Tên món ăn'].iloc[0])

In [20]:
def get_average_rating(userid):
    rating_user = train_rating.loc[((train_rating.UserID== userid)), 'rating']
    return np.mean(rating_user)

Các hàm tính độ tương tự 

In [21]:
#Độ đo Cosine

def cosine_similarity_user(user1, user2):
    both_watch_count = 0
    for foodid in train_rating.loc[train_rating.UserID == user1, 'FoodId'].to_list():
        for foodid in train_rating.loc[train_rating.UserID == user2, 'FoodId'].tolist():
            both_watch_count +=1
    if both_watch_count == 0:
        return 0

    rating1 = []
    rating2 = []

    for foodid in train_rating.loc[train_rating.UserID == user1, 'FoodId'].to_list():
        if foodid in train_rating.loc[train_rating.UserID == user2, 'FoodId'].tolist():
            rating1.append(get_rating(user1, foodid))
            rating2.append(get_rating(user2, foodid))

    if (norm(rating1)*norm(rating2)) == 0:
        return 0
    return dot(rating1, rating2)/(norm(rating1)*norm(rating2))

In [22]:
# Độ đo tương tự Peason

def pearson_correlation_user(user1, user2):
    both_watch_count = []
    for foodid in train_rating.loc[(train_rating.UserID == user1), 'FoodId'].tolist():
        if foodid in train_rating.loc[(train_rating.UserID == user2), 'FoodId'].tolist():
            both_watch_count.append(foodid)
    if len(both_watch_count) == 0:
        return 0
    rating_sum_1 = sum([get_rating(user1, foodid) for foodid in both_watch_count])
    avg_rating_sum_1 = rating_sum_1/len(both_watch_count)

    rating_sum_2 = sum([get_rating(user2, foodid) for foodid in both_watch_count])
    avg_rating_sum_2 = rating_sum_2/len(both_watch_count)

    #Gia tri tren tu
    numerator = sum([(get_rating(user1, foodid)- avg_rating_sum_1)*(get_rating(user2, foodid) - avg_rating_sum_2) for foodid in both_watch_count])
    denominator = sqrt(sum([pow((get_rating(user1, foodid) - avg_rating_sum_1), 2) for foodid in both_watch_count]))*sqrt(sum([pow((get_rating(user2, foodid) - avg_rating_sum_2), 2) for foodid in both_watch_count]))
    if denominator == 0:
        return 0
    return numerator/denominator

In [23]:
# Hàm tính top k người dùng có độ tương đồng cao nhất

def get_similarity_user(user1, number_of_user, metric_similar):
    user_ids = train_rating.UserID.unique().tolist()

    similarity_score = []
    for userID in user_ids:
        if (userID != user1):
            if (metric_similar == "cosine"):
                similarity_score.append((cosine_similarity_user(user1, userID), userID))
            elif (metric_similar == "pearson"):
                similarity_score.append((pearson_correlation_user(user1, userID), userID))
    similarity_score.sort()
    similarity_score.reverse()
    return similarity_score[:number_of_user]

In [24]:
# Khuyến nghị dựa trên trung bình đánh giá

def get_recommendation_tbdg_user(userid, number_of_user, metric_similar):
    user_ids = train_rating.UserID.unique().tolist()
    total = {}
    number_rating = {}
    similarity_sum = {}
    List_similar = get_similarity_user(userid, number_of_user, metric_similar)
    for peason, user in List_similar:
        for foodid in get_foodid(user):
            if foodid not in get_foodid(userid) or get_rating(userid, foodid) == 0:
                if foodid not in total:
                    total[foodid] = 0
                    number_rating[foodid] = 0
                    total[foodid] += get_rating(user, foodid)
                    if get_rating(user, foodid) != 0:
                        number_rating[foodid] +=1
                else:
                    total[foodid] += get_rating(user, foodid)
                    if get_rating(user, foodid) != 0:
                        number_rating[foodid] +=1
   
    ranking = [(to/number_rating[foodid], foodid) for foodid, to in total.items()]
    ranking.sort()
    ranking.reverse()

    recomendations = [(get_food_title(foodid), score) for score, foodid in ranking]

    return recomendations

In [25]:
# Khuyến nghị dựa theo trung bình đánh giá có trọng số

def get_recommendation_tbdgcts_user(userid, number_of_user, metric_similar):
    user_ids = train_rating.UserID.unique().tolist()
    total = {}
    similarity_sum = {}
    List_similar = get_similarity_user(userid, number_of_user, metric_similar)
    for peason, user in List_similar:
        score = peason
        for foodid in get_foodid(user):
            if foodid not in get_foodid(userid) or get_rating(userid, foodid) == 0:
                if foodid not in total:
                    total[foodid] = 0
                    total[foodid] += get_rating(user, foodid)*score
                    similarity_sum [foodid] = 0
                    similarity_sum [foodid] +=score
                else:
                    total[foodid] += get_rating(user, foodid)*score
                    similarity_sum [foodid] +=score
                
    ranking = [(to/similarity_sum[foodid], foodid) for foodid, to in total.items()]
    ranking.sort()
    ranking.reverse()

    recomendations = [(get_food_title(foodid), score) for score, foodid in ranking]

    return recomendations

In [26]:
# Khuyến nghị dựa trên khoảng đánh giá

def get_recommendation_kdg_user(userid, number_of_user, metric_similar):
    user_ids = train_rating.UserID.unique().tolist()
    total = {}
    similarity_sum = {}
    List_similar = get_similarity_user(userid, number_of_user, metric_similar)
    for peason, user in List_similar:
        score = peason
        for foodid in get_foodid(user):
            if foodid not in get_foodid(userid) or get_rating(userid, foodid) == 0:
                if foodid not in total:
                    total[foodid] = 0
                    total[foodid] += (get_rating(user, foodid) - get_average_rating(user))*score
                    similarity_sum [foodid] = 0
                    similarity_sum [foodid] +=score
                else:
                    total[foodid] += (get_rating(user, foodid) - get_average_rating(user))*score
                    similarity_sum [foodid] +=score
                
    ranking = [((get_average_rating(userid) + to/similarity_sum[foodid]), foodid) for foodid, to in total.items()]
    ranking.sort()
    ranking.reverse()

    recomendations = [(get_food_title(foodid), score) for score, foodid in ranking]

    return recomendations

In [27]:
# Khuyến nghị món ăn

def get_recommendation_user(userid, number_of_user, metrics_rating, metric_similar):
    rated_food = train_rating[train_rating['UserID'] == userid].FoodId.values
    if len(rated_food) <= 2:
        recomendations = average_rating_recommend(rated_food)
    else:
        if (metrics_rating == "tbdg"):
            recomendations = get_recommendation_tbdg_user(userid, number_of_user, metric_similar)
            return recomendations
        elif (metrics_rating == "tbdgcts"):
            recomendations = get_recommendation_tbdgcts_user(userid, number_of_user, metric_similar)
            return recomendations
        elif (metrics_rating == "kdg"):
            recomendations = get_recommendation_kdg_user(userid, number_of_user, metric_similar)
    recomendations = sorted(recomendations.items(), key=lambda x: x[1], reverse=True)

    return recomendations

# Catboost

In [28]:
def predict_catboost(user_id):
    with open("/content/drive/MyDrive/Food Recommend Project/Linh Models for redcommend/catboost.pkl", 'rb') as model_file:
        model = pickle.load(model_file)
    rated_food = train_rating[train_rating['UserID'] == user_id].FoodId.values
    
    if len(rated_food) <= 2:
        score_dict = average_rating_recommend(rated_food)
    else:
        score_dict = {}
        context_user = get_user_context(user_id)
        
        for i in range(food_feature.shape[0]):
            if food_feature.loc[i, 'FoodID'] in rated_food:
                continue
            else:
                feature_vector = context_user.values.tolist()[0] + food_feature.iloc[i, 1:].tolist() + tf_idf_matrix[i].tolist()

                score = model.predict(feature_vector)
                food_id = food_feature.loc[i, 'FoodID']
                score_dict[get_food_name(food_id)] = score

    score_dict = sorted(score_dict.items(), key=lambda x: x[1], reverse=True)
    return score_dict

# Neural network model

In [29]:
def predict_neural(user_id):
    model = tf.keras.models.load_model('/content/drive/MyDrive/Food Recommend Project/Linh Models for redcommend/neural.h5')
    rated_food = train_rating[train_rating['UserID'] == user_id].FoodId.values

    if len(rated_food) <= 2:
        score_dict = average_rating_recommend(rated_food)
    else:
        score_dict = {}
        context_user = get_user_context(user_id)
        for i in range(food_feature.shape[0]):
            if food_feature.loc[i, 'FoodID'] in rated_food:
                continue
            else:
                a = np.asarray(pd.DataFrame(context_user.values[0]).transpose()).astype(np.float32)
                b = np.asarray(pd.DataFrame(food_feature.iloc[i, 1:]).transpose()).astype(np.float32)
                c = np.asarray(pd.DataFrame(tf_idf_matrix[i]).transpose()).astype(np.float32)
                score = model.predict([a, b, c])
                score = score[0][0]
                food_id = food_feature.loc[i, 'FoodID']
                score_dict[get_food_name(food_id)] = score

    score_dict = sorted(score_dict.items(), key=lambda x: x[1], reverse=True)
    return score_dict

# Neural Collaborative Filtering

In [30]:
def predict_neural_cf(user_id):
    model = tf.keras.models.load_model('/content/drive/MyDrive/Food Recommend Project/Linh Models for redcommend/neural CF new test.h5')
    rated_food = train_rating[train_rating['UserID'] == user_id].FoodId.values

    if len(rated_food) <= 2:
        score_dict = average_rating_recommend(rated_food)
    else:
        score_dict = {}
        for i in range(food_feature.shape[0]):
            if food_feature.loc[i, 'FoodID'] in rated_food:
                continue
            else:
                a = pd.Series(float(user_id[1:]))
                b = pd.Series(float(food_feature.loc[i, 'FoodID'][1:]))
                
                score = model.predict([a, b])
                score = score[0][0]
                food_id = food_feature.loc[i, 'FoodID']
                score_dict[get_food_name(food_id)] = score

    score_dict = sorted(score_dict.items(), key=lambda x: x[1], reverse=True)
    return score_dict

# -=-=-=-=-=-=-Final predict-=-=-=-=-=-=-#

In [31]:
def predict():
    print('Nhập id của user cần khuyến nghị: ')
    user_id = input()
    print('Nhập loại thuật toán (catboost, neural, neural_cf hoặc cf): ')
    model_type = ''
    while (model_type not in ['catboost', 'neural', 'neural_cf', 'cf']):
        model_type = input()
    if model_type == 'catboost':
        predict_dict = predict_catboost(user_id)
    elif model_type == 'neural':
        predict_dict = predict_neural(user_id)
    elif model_type == 'neural_cf':
        predict_dict = predict_neural_cf(user_id)
    elif model_type == 'cf':
        print('Nhập top K người dùng tương tự (đề xuất: 10): ')
        try:
            number_of_user = int(input())
        except:
            number_of_user = 10
        print('Nhập phương pháp dự đoán rating ("tbdg", "tbdgcts" hoặc "kdg"): ')
        metrics_rating = ''
        while (metrics_rating not in ['tbdg', 'tbdgcts', 'kdg']):
            metrics_rating = input()
        print('Nhập phương pháp tính toán độ tương tự ("cosine" hoặc "pearson"): ')
        metric_similar = ''
        while (metric_similar not in ['cosine', 'pearson']):
            metric_similar = input()
        predict_dict = get_recommendation_user(user_id, number_of_user, metrics_rating, metric_similar)
    print('===============================')
    print('Danh sách các món ăn đề xuất: \n')
    return predict_dict

# DEMO

In [32]:
predict()

Nhập id của user cần khuyến nghị: 
U36
Nhập loại thuật toán (catboost, neural, neural_cf hoặc cf): 
cf
Nhập top K người dùng tương tự (đề xuất: 10): 
11
Nhập phương pháp dự đoán rating ("tbdg", "tbdgcts" hoặc "kdg"): 
tbdg
Nhập phương pháp tính toán độ tương tự ("cosine" hoặc "pearson"): 
cosine
Danh sách các món ăn đề xuất: 



[('Phở bò', 4.333333333333333),
 ('Bún bò', 4.0),
 ('Bún thịt nướng', 4.0),
 ('Hủ tiếu', 3.8333333333333335),
 ('Mì quảng', 3.75),
 ('Bánh bao', 3.7142857142857144),
 ('Bò kho', 3.6),
 ('Gà nướng cơm lam', 3.4285714285714284),
 ('Gỏi gà', 3.3333333333333335),
 ('Súp cua', 3.25),
 ('Bún mắm', 3.090909090909091),
 ('Cơm tấm', 3.0),
 ('Cháo lòng', 2.857142857142857),
 ('Bún riêu', 2.75),
 ('Bánh bèo', 2.5)]